<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale and Batch Processing:<br>Apache Spark on Cloud Pak for Data with IBM Analytics Engine

This notebook must be run in the Python 3.10 runtime environment. It requires Watson OpenScale service credentials.

The notebook configures Watson OpenScale to monitor any WML online deployment as a self managed subscription on Openscale. For self managed subscriptions, all the monitors are run as jobs on user provided spark engine. Use the notebook to enable quality, drift, fairness and explainability monitoring and run on-demand evaluations. Before you can run the notebook, you must have the following resources:

1. Configuration Archive generated using common configuration notebook [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb) and pre created WML deployment.
2. Payload, feedback, drifted transactions, explanations queue and result tables to be created or validated in an Apache Hive datawarehouse. Please note, some of these maybe optional depending on your use-case.

## Contents

* [1. Setup](#setup)
* [2. Configure Watson OpenScale](#openscale)
* [3. Set up Subscription](#subscription)
* [4. Quality monitoring](#quality)
* [5. Drift monitoring](#drift)
* [6. Fairness monitoring](#fairness)
* [7. Explainability monitoring](#explainability)

# 1. Setup <a name="setup"></a>

### Installing Required Libraries

First import some of the packages you need to use. After you finish installing the following software packages, restart the kernel.

### Import configuration archive/package

Configuration archive/package created using configuration notebook will be required to onboard model for monitoring in IBM Watson OpenScale. Provide path location of archive here.

Please note if you are executing this notebook in IBM Watson Studio, first upload the configuration archive/package to project and use provided code snippet to download it to local directory of this notebook.

In [1]:
import warnings
import tarfile
warnings.filterwarnings("ignore")
%env PIP_DISABLE_PIP_VERSION_CHECK=1

# Note: Restart kernel after the dependencies are installed
!pip install --upgrade ibm-watson-openscale
!pip install "ibm_wos_utils~=5.0.0"

# # Download "configuration_archive.tar.gz" from project to local directory and 
# extract the artifacts
# from ibm_watson_studio_lib import access_project_or_space
# wslib = access_project_or_space()
# wslib.download_file("configuration_archive.tar.gz")
archive_file_path = "configuration_archive.tar.gz"
with tarfile.open('configuration_archive.tar.gz', "r:gz") as tar:
    tar.extractall('.')
# On unpacking the configuration archive, the files [common_configuration.json, drift_archive.tar.gz, 
# explainability.tar.gz, fairness_statistics.json] will be extracted to the current directory. 
# Output may differ depending on which monitors were enabled while running the 
# common configuration notebook

env: PIP_DISABLE_PIP_VERSION_CHECK=1


In [2]:
!pip show ibm-watson-openscale

Name: ibm-watson-openscale
Version: 3.0.39
Summary: Client library for IBM Watson OpenScale
Home-page: https://github.ibm.com/watson-developer-cloud/openscale-python-sdk
Author: IBM Watson OpenScale
Author-email: kishore.patel@in.ibm.com
License: Apache 2.0
Location: /opt/conda/envs/Python-RT24.1-Premium/lib/python3.11/site-packages
Requires: ibm-cloud-sdk-core, pandas, python-dateutil, requests
Required-by: 


## Configure credentials

Provide your IBM Watson OpenScale and WML credentials in the following cell:

In [3]:
WOS_CREDENTIALS = {
    "url": "<cluster-url>",
    "username": "<username>",
    "password": "<password>",
    "instance_id": "<openscale instance id>"
}

WML_CREDENTIALS = {
    "url": "<wml_url>",
    "username": "<username",
    "password": "<password>",
    "instance_id": "<instance id>"
}

## Specify model details

### Service provider and subscription metadata

In [4]:
# Service Provider

SERVICE_PROVIDER_NAME = "<service-provider-name>"
SERVICE_PROVIDER_DESCRIPTION = "<service-provider-description>"

# Subscription

SUBSCRIPTION_NAME = "<subscription-name>"
SUBSCRIPTION_DESCRIPTION = "<subscription-description>"

### IBM Analytics Engine - Spark

Make sure that the Apache Spark manager on IBM Analytics Engine is running, and then provide the following details:

- IAE_SPARK_DISPLAY_NAME: _Display Name of the Spark instance in IBM Analytics Engine_
- IAE_SPARK_JOBS_ENDPOINT: _Spark Jobs Endpoint for IBM Analytics Engine_
- IBM_CPD_VOLUME: _IBM Cloud Pak for Data storage volume name_
- IBM_CPD_USERNAME: _IBM Cloud Pak for Data username_
- IBM_CPD_APIKEY: _IBM Cloud Pak for Data API key_
- IAE_SPARK_DESCRIPTION: _Custom description for the Spark instance_

In [5]:
IAE_SPARK_DISPLAY_NAME = "<spark-engine-name>"
IAE_SPARK_JOBS_ENDPOINT = "<spark-job-endpoint-for-ibm-analytics-engine>"
IBM_CPD_VOLUME = "<ibm-cpd-volume>"
IBM_CPD_USERNAME = "<ibm-cloud-pak-for-data-username>"
IBM_CPD_APIKEY = "<ibm-cloud-pak-for-data-apikey>"
IAE_SPARK_NAME = "<iae-spark-name>"
IAE_SPARK_DESCRIPTION = "<iae-spark-description>"

#### Provide Spark Resource Settings

To configure how much of your Spark Cluster resources this job can consume, edit the following values:

- max_num_executors: _Maximum Number of executors to launch for this session_
- min_executors: _Minimum Number of executors to launch for this session_
- executor_cores: _Number of cores to use for each executor_   
- executor_memory: _Amount of memory (in GBs) to use per executor process_
- driver_cores: _Number of cores to use for the driver process_
- driver_memory: _Amount of memory (in GBs) to use for the driver process_

In [6]:
spark_parameters = {
    "max_num_executors": 1,
    "min_num_executors": 1,
    "executor_cores": 1,
    "executor_memory": 1,
    "driver_cores": 1,
    "driver_memory": 1
}

### Apache Hive

To connect to Apache Hive, you must provide the following details:

- HIVE_CONNECTION_NAME: _Custom display name for the Hive Connection_
- HIVE_CONNECTION_DESCRIPTION: _Custom description for the Hive connection_
- HIVE_METASTORE_URI: _Thrift URI for Hive Metastore to connect to_

In case of a kerberos-enabled hive, provide additional details required to obtain the Hadoop delegation token:
- HIVE_KERBEROS_PRINCIPAL: The kerberos principal used to generate the delegation token
- DELEGATION_TOKEN_SECRET_URN: The secret_urn of the CP4D vault where the token is stored *OR*
- DELEGATION_TOKEN_ENDPOINT: The REST endpoint which generates and returns the delegation token

In [7]:
HIVE_CONNECTION_NAME = "<hive-connection-name>"
HIVE_CONNECTION_DESCRIPTION = "<hive-connection-description>"
HIVE_METASTORE_URI = "<hive-metastore-uri>"

# Flag to indicate if the Hive is secured with kerberos
KERBEROS_ENABLED = False
# Provide Hadoop delegation token details if KERBEROS_ENABLED is True
# Provide either secret_urn of the CP4D vault OR the delegation token endpoint. One of the two fields is mandatory to fetch the delegation token.
HIVE_KERBEROS_PRINCIPAL = "<The kerberos principal used to generate the delegation token>"
DELEGATION_TOKEN_SECRET_URN = "<The secret_urn of the CP4D vault where the token is stored>"
DELEGATION_TOKEN_ENDPOINT = "<The REST endpoint which generates and returns the delegation token>"

### Feedback table metadata

The quality monitor stores metadata in the feedback table. To configure the quality monitor, you must provide the following details. To skip quality monitoring, run the following cell to initialize variables with the value of `None`.

- FEEDBACK_DATABASE_NAME: _Database name where feedback table is present_
- FEEDBACK_SCHEMA_NAME: _Schema name where feedback table is present_
- FEEDBACK_TABLE_NAME: _Name of the feedback table_

In [8]:
#feedback

FEEDBACK_DATABASE_NAME = None
FEEDBACK_TABLE_NAME = None

### Payload and drift table metadata

The drift monitor stores metadata in the payload and drift tables. To configure the drift monitor, you must provide the following details. To skip drift monitoring, run the following cell to initialize variables with the value of `None`.

- PAYLOAD_DATABASE_NAME: _Database name where payload logging table is present_
- PAYLOAD_SCHEMA_NAME: _Schema name where payload logging table is present_
- PAYLOAD_TABLE_NAME: _Name of the payload logging table_
- DRIFT_DATABASE_NAME: _Database name where drifted transactions table is present_
- DRIFT_SCHEMA_NAME: _Schema name where drifted transactions table is present_
- DRIFT_TABLE_NAME: _Name of the drifted transactions table_

In [9]:
#payload logging

PAYLOAD_DATABASE_NAME = None
PAYLOAD_TABLE_NAME = None

#drift

DRIFT_DATABASE_NAME = None
DRIFT_TABLE_NAME = None

### Explainability table metadata

The explainability monitor requires the queue and result tables. The payload table can also be used as the queue table. To configure the explainability monitor, you must provide the following details. To skip explainability monitoring, run the following cell to initialize variables with the value of `None`.

- EXPLAINABILITY_DATABASE_NAME: _Database name where explanations queue, result tables are present_
- EXPLAINABILITY_QUEUE_TABLE_NAME: _Name of the explanations queue table_
- EXPLAINABILITY_RESULT_TABLE_NAME: _Name of the explanations result table_

In [10]:
#explainability

EXPLAINABILITY_DATABASE_NAME = None
EXPLAINABILITY_QUEUE_TABLE_NAME = None
EXPLAINABILITY_RESULT_TABLE_NAME = None

# 2. Configure Watson OpenScale <a name="openscale"></a>

### Import the required libraries and set up the Watson OpenScale client

In [11]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *

authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS["url"],
        username=WOS_CREDENTIALS["username"],
        password=WOS_CREDENTIALS["password"],
        disable_ssl_verification=True
    )

wos_client = APIClient(authenticator=authenticator, service_url=WOS_CREDENTIALS["url"], service_instance_id=WOS_CREDENTIALS["instance_id"])
data_mart_id=WOS_CREDENTIALS["instance_id"]

### Display Watson OpenScale datamart details

In [12]:
wos_client.data_marts.show()

AIOSFASTPATHICP-00000000-0000-0000-0000-000000000000,Data Mart created by OpenScale ExpressPath,False,active,2024-06-04 05:19:03.698000+00:00,00000000-0000-0000-0000-000000000000


### Create a service provider

In [13]:
# Delete existing service provider with the same name as provided

service_providers = wos_client.service_providers.list().result.service_providers
for provider in service_providers:
    if provider.entity.name == SERVICE_PROVIDER_NAME:
        wos_client.service_providers.delete(service_provider_id=provider.metadata.id)
        break

In [14]:
# Add Service Provider

added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        credentials=WML_CREDENTIALS,
        operational_space_id="production",
        background_mode=False
    ).result

service_provider_id = added_service_provider_result.metadata.id

wos_client.service_providers.show()




 Waiting for end of adding service provider e7e9f4fe-5a9d-4ac8-b284-8bc3dabb749b 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




00000000-0000-0000-0000-000000000000,active,WML_IAE6,watson_machine_learning,2024-07-17 06:01:27.589000+00:00,e7e9f4fe-5a9d-4ac8-b284-8bc3dabb749b
None,active,Custom ML Provider Demo - All Monitors,custom_machine_learning,2024-07-15 06:22:52.902000+00:00,812592fa-7a84-444e-8f1f-6891be056bcf
99999999-9999-9999-9999-999999999999,active,Image Binary WML V2_test,watson_machine_learning,2024-07-09 14:59:27.678000+00:00,0f363199-46fd-496e-8d92-e83129583ab4
None,active,RC - OpenScale Headless Service Provider,custom_machine_learning,2024-07-09 13:44:40.294000+00:00,76b7ac24-760c-4682-b6c9-c2b20bafc39b
99999999-9999-9999-9999-999999999999,active,GCR Auto AI prod,watson_machine_learning,2024-07-04 06:42:11.028000+00:00,43063a84-b760-479c-a2ca-bc5d60f929fd
99999999-9999-9999-9999-999999999999,active,GCR AutoAI space Demo,watson_machine_learning,2024-07-03 14:31:13.702000+00:00,0dcfefc6-3b44-4387-a2e1-b1a693f38f62
None,active,IAE7,custom_machine_learning,2024-07-02 09:08:06.273000+00:00,184e73a2-7fd8-4f3f-b994-bb648f6eb8ec
None,active,IAE6,custom_machine_learning,2024-07-02 09:04:22.643000+00:00,644befcd-6d36-4f4d-a30a-cd51a28b63fe
None,active,WML_IAE5,custom_machine_learning,2024-07-02 08:43:08.723000+00:00,e986a0d7-8187-4fed-ab2a-c614a9683cae
None,active,WML_IAE4,custom_machine_learning,2024-07-02 07:00:41.816000+00:00,d90c6bf2-49c6-4179-9876-8b85b0247d95


Note: First 10 records were displayed.


In [15]:
service_provide_details = wos_client.service_providers.get(service_provider_id=service_provider_id).result
print(service_provide_details)

{
  "metadata": {
    "id": "e7e9f4fe-5a9d-4ac8-b284-8bc3dabb749b",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:service_provider:e7e9f4fe-5a9d-4ac8-b284-8bc3dabb749b",
    "url": "/v2/service_providers/e7e9f4fe-5a9d-4ac8-b284-8bc3dabb749b",
    "created_at": "2024-07-17T06:01:27.589000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "WML_IAE6",
    "service_type": "watson_machine_learning",
    "instance_id": "00000000-0000-0000-0000-000000000000",
    "credentials": {
      "secret_id": "f080ea35-b89f-4c82-bfa3-fe3c9aa70c25"
    },
    "operational_space_id": "production",
    "status": {
      "state": "active"
    }
  }
}


### Create integrated systems for Spark Engine and Hive

In [16]:
# Delete existing spark and hive integrated systems if present

integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems

for system in integrated_systems:
    if system.entity.name in (IAE_SPARK_NAME, HIVE_CONNECTION_NAME):
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

Deleting integrated system WML_IAEKB_Spark
Deleting integrated system WML_IAEKB_Spark
Deleting integrated system Hive_WML_IAE_temp
Deleting integrated system Hive_WML_IAE_temp


#### Spark Engine

In [17]:
spark_integrated_system = IntegratedSystems(wos_client).add(
    name=IAE_SPARK_NAME,
    description=IAE_SPARK_DESCRIPTION,
    type="spark",
    credentials={
        "username": IBM_CPD_USERNAME,
        "apikey": IBM_CPD_APIKEY
    },
    connection={
        "display_name": IAE_SPARK_DISPLAY_NAME,
        "endpoint": IAE_SPARK_JOBS_ENDPOINT,
        "volume": IBM_CPD_VOLUME,
        "location_type": "cpd_iae"
    }
).result

spark_integrated_system_id = spark_integrated_system.metadata.id
print(spark_integrated_system)

{
  "metadata": {
    "id": "5d8aa22c-1e5d-43f9-9594-d7a0c5bf704f",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:5d8aa22c-1e5d-43f9-9594-d7a0c5bf704f",
    "url": "/v2/integrated_systems/5d8aa22c-1e5d-43f9-9594-d7a0c5bf704f",
    "created_at": "2024-07-17T06:01:36.601000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "WML_IAEKB_Spark",
    "type": "spark",
    "description": "WML_IAEKB_Spark",
    "credentials": {
      "secret_id": "6c6d4a1a-cbdc-4aa7-a6e7-46b953f34eb3"
    },
    "connection": {
      "display_name": "IAEBatchSpark",
      "endpoint": "https://cpd-cpd-instance.apps.wos415nfs2672.cp.fyre.ibm.com/v4/analytics_engines/7d2a4875-563e-42d0-994f-518ebf6e1e42/spark_applications",
      "location_type": "cpd_iae",
      "volume": "cpd-instance::IAEBatchTest"
    }
  }
}


#### Hive

In [18]:
hive_connection = {}
hive_credentials = {}
if HIVE_METASTORE_URI is not None:
    hive_connection["metastore_url"] = HIVE_METASTORE_URI
    hive_connection["location_type"] = "metastore"

if KERBEROS_ENABLED is True:
    hive_connection["kerberos_enabled"] = True
    hive_credentials["kerberos_principal"] = HIVE_KERBEROS_PRINCIPAL
    if DELEGATION_TOKEN_SECRET_URN:
        hive_credentials["delegation_token_urn"] = DELEGATION_TOKEN_SECRET_URN
    if DELEGATION_TOKEN_ENDPOINT:
        hive_credentials["delegation_token_endpoint"] = DELEGATION_TOKEN_ENDPOINT

hive_integrated_system = IntegratedSystems(wos_client).add(
    name=HIVE_CONNECTION_NAME,
    description=HIVE_CONNECTION_DESCRIPTION,
    type="hive",
    credentials=hive_credentials,
    connection=hive_connection
).result

hive_integrated_system_id=hive_integrated_system.metadata.id
print(hive_integrated_system)

{
  "metadata": {
    "id": "3003e604-c49a-4bd5-829e-a4fe2602152d",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:3003e604-c49a-4bd5-829e-a4fe2602152d",
    "url": "/v2/integrated_systems/3003e604-c49a-4bd5-829e-a4fe2602152d",
    "created_at": "2024-07-17T06:01:38.889000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "Hive_WML_IAE_temp",
    "type": "hive",
    "description": "Hive_WML_IAE_temp",
    "credentials": {
      "secret_id": "720a2d2c-37fb-499e-9893-ccdc913a9706"
    },
    "connection": {
      "location_type": "metastore",
      "metastore_url": "thrift://shillong1.fyre.ibm.com:9083"
    }
  }
}


# 3. Set up a subscription<a name="subscription"></a>

In [19]:
# Delete an existing subscription with the provided name

subscriptions = wos_client.subscriptions.list().result.subscriptions
for sub in subscriptions:
    if sub.entity.deployment.name == SUBSCRIPTION_NAME:
        wos_client.subscriptions.delete(subscription_id=sub.metadata.id)
        break

# Display all subscriptions
wos_client.subscriptions.show()

5e068176-72b5-4931-ac79-9da5bab5acff,model,[asset] Custom ML Subscription - All Monitors,00000000-0000-0000-0000-000000000000,9304aa50-a77f-414c-a985-fa32f3449002,[asset] Custom ML Subscription - All Monitors,812592fa-7a84-444e-8f1f-6891be056bcf,active,2024-07-15 06:23:47.712000+00:00,df2236c1-e96d-436c-a631-113a5f80cbcd
172ef5c8-096b-4133-b159-f3d3c5586e5b,model,Dog-Cat binary,00000000-0000-0000-0000-000000000000,c99073b2-daab-4bb0-9fa8-82d97fb47f89,Dog-Cat binary deployment,0f363199-46fd-496e-8d92-e83129583ab4,active,2024-07-09 15:00:41.072000+00:00,775afc50-ff8a-4846-9db2-6dcdc9916ae6
25a7e048-2e21-4aec-b035-1863853951ab,model,[asset] RC - GCR Headless Subscription,00000000-0000-0000-0000-000000000000,aa6d890c-9ee6-4d54-aa8e-bb94b54daab0,[asset] RC - GCR Headless Subscription,76b7ac24-760c-4682-b6c9-c2b20bafc39b,active,2024-07-09 13:46:10.011000+00:00,2d6b0008-6d7a-4830-9fb0-cb938dacbbb4
91ad45c3-0bda-4ea3-a35f-7c814f7987ad,model,GCR AutoAI Demo - P4 Snap Random Forest Classifier - Model,00000000-0000-0000-0000-000000000000,5382d5ef-41f5-44b4-994a-8bd1d948b35e,GCR Auto AI prod,43063a84-b760-479c-a2ca-bc5d60f929fd,active,2024-07-04 06:42:29.105000+00:00,dd9b03e6-fecd-47dc-b277-7d8545220c5d
19754024-651a-4eff-a3e6-044e6c61867a,model,GCR AutoAI Demo - P4 Snap Random Forest Classifier - Model,00000000-0000-0000-0000-000000000000,452fdeb6-7b59-4f8f-b3f7-4ef8421c92cf,GCR AutoAI Demo,0dcfefc6-3b44-4387-a2e1-b1a693f38f62,active,2024-07-03 15:19:39.096000+00:00,bbecb696-24aa-489b-9f4a-0fd70cd50214
e3ac9fc3-bccf-4a4e-b37b-490bfb93dd81,model,GCR AutoAI - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,6399e6e8-df5a-4370-9af4-34b2f2e76bc6,GCR Auto AI,a7ca157a-de07-457a-8c4c-b1a2e998699c,active,2024-07-03 10:37:20.487000+00:00,ce36911c-75f6-4f99-b4d2-b71e5d55a802
592b902d-3dc9-4e56-8bcb-86cbf1a6d8a9,model,gcr - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,2b976af0-e4ab-4859-af7d-2f2287d864ad,gcr model,4d2f2fb2-6b64-4d58-8f13-257166e468e9,active,2024-07-02 15:33:17.454000+00:00,e96278a6-7190-48f3-b8bd-945fa48cfe50
327d8aea-ecfc-4990-9bb9-601a1695094d,model,GCR AutoAI - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,755c3e75-24b5-4839-8a8f-3f85c07a40c9,GCR demo,a7ca157a-de07-457a-8c4c-b1a2e998699c,active,2024-07-02 12:03:14.224000+00:00,a0f86241-8bfc-4322-895a-2597512e1653
b21904ef-7478-4dae-b93f-c120e95c9200,model,My SDK Batch Subscription-db2,00000000-0000-0000-0000-000000000000,a10a121b-2394-4fe7-9a2d-f0520abd212c,My SDK Batch Subscription-db2,644befcd-6d36-4f4d-a30a-cd51a28b63fe,active,2024-07-02 09:04:38.883000+00:00,70c9c394-2530-4c0f-b0b2-6b81e44612d0
7bf1d492-3275-405d-9b61-da4e2075e746,model,My SDK Batch Subscription-db2,00000000-0000-0000-0000-000000000000,a302d0d4-3f38-4ded-8688-3e4b2e9bb55c,My SDK Batch Subscription-db2,e986a0d7-8187-4fed-ab2a-c614a9683cae,active,2024-07-02 08:43:25.526000+00:00,f4178567-4f9a-4b0e-b255-2babdb8b5f38


Note: First 10 records were displayed.


### Set subscription metadata

In the following cell, we expect you extracted Configuration Archive to current directory and it contains common_configuration.json. If not true, please update path to correct location. After you edit the path information, run the cell to set the asset details and properties, the deployment details, the analytics engine details, and to add the required tables as data sources.

In [20]:
import uuid
import json
# Provide the WML deployment io and space d
deployment_uid = "<wml_deployment_id>"
space_id = "<wml_space_id>"
common_configuration = None

with open("common_configuration.json", "r") as fp:
    configuration_json = json.load(fp)
    common_configuration = configuration_json.get("common_configuration")
    if common_configuration is None:
        raise Exception("Please provide the correct path to the common configuration JSON")
        
asset_deployment_details = wos_client.service_providers.list_assets(
    data_mart_id=data_mart_id, service_provider_id=service_provider_id, 
    deployment_id = deployment_uid, deployment_space_id = space_id).result['resources'][0]

model_asset_details_from_deployment = wos_client.service_providers.get_deployment_asset(
    data_mart_id=data_mart_id,service_provider_id=service_provider_id,deployment_id=deployment_uid,
    deployment_space_id=space_id)

scoring_endpoint = ScoringEndpointRequest(
    url=model_asset_details_from_deployment['entity']['scoring_endpoint']['url'] )
    
# Set asset details
asset = Asset(
    asset_id=model_asset_details_from_deployment["entity"]["asset"]["asset_id"],
    url=model_asset_details_from_deployment["entity"]["asset"]["url"],
    name=SUBSCRIPTION_NAME,
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=common_configuration.get("problem_type") if common_configuration.get("problem_type") else common_configuration.get("model_type")
)

# Set deployment details
asset_deployment = AssetDeploymentRequest(
    deployment_id=asset_deployment_details['metadata']['guid'],
    name=SUBSCRIPTION_NAME,
    description=SUBSCRIPTION_DESCRIPTION,
    deployment_type=DeploymentTypes.ONLINE,
    scoring_endpoint=scoring_endpoint
)

probability_fields = common_configuration.get("probability_fields") if common_configuration.get("probability_fields") else [common_configuration.get("probability")]


# Set asset properties 
asset_properties_request = AssetPropertiesRequest(
    label_column=common_configuration["label_column"],
    probability_fields=probability_fields, # comment out this line for regression models as probability_fields is not applicable
    prediction_field=common_configuration["prediction"],
    feature_fields=common_configuration["feature_columns"],
    categorical_fields=common_configuration["categorical_columns"]
)

{
  "url": "https://internal-nginx-svc:12443/ml/v4/deployments/2b976af0-e4ab-4859-af7d-2f2287d864ad/predictions"
}


In [21]:
# Set analytics engine details
analytics_engine = AnalyticsEngine(
    type="spark",
    integrated_system_id=spark_integrated_system_id,
    parameters = spark_parameters
)

# Add selected tables as data sources
data_sources = []
if FEEDBACK_DATABASE_NAME is not None and FEEDBACK_TABLE_NAME is not None:
    feedback_data_source = DataSource(
        type="feedback", 
        database_name=FEEDBACK_DATABASE_NAME,
        schema_name=FEEDBACK_DATABASE_NAME,
        table_name=FEEDBACK_TABLE_NAME,
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_integrated_system_id
        ),
        parameters={
            "hive_storage_format": "<to_be_edited>" #supported values are "csv", "parquet", "orc"
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    data_sources.append(feedback_data_source)
    
if PAYLOAD_DATABASE_NAME is not None and PAYLOAD_TABLE_NAME is not None \
    and DRIFT_DATABASE_NAME is not None and DRIFT_TABLE_NAME is not None:
    payload_logging_data_source = DataSource(
        type="payload", 
        database_name=PAYLOAD_DATABASE_NAME,
        schema_name=PAYLOAD_DATABASE_NAME,
        table_name=PAYLOAD_TABLE_NAME,
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_integrated_system_id
        ),
        parameters={
            "hive_storage_format": "<to_be_edited>" #supported values are "csv", "parquet", "orc"
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    drifted_transactions_table_data_source = DataSource(
        type="drift", 
        database_name=DRIFT_DATABASE_NAME,
        schema_name=DRIFT_DATABASE_NAME,
        table_name=DRIFT_TABLE_NAME,
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_integrated_system_id
        ),
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(payload_logging_data_source)
    data_sources.append(drifted_transactions_table_data_source)

if EXPLAINABILITY_DATABASE_NAME is not None and \
    EXPLAINABILITY_QUEUE_TABLE_NAME is not None and \
        EXPLAINABILITY_RESULT_TABLE_NAME is not None:

    explainability_queue_data_source = DataSource(
        type="explain_queue", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_DATABASE_NAME,
        table_name=EXPLAINABILITY_QUEUE_TABLE_NAME,
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_integrated_system_id
        ),
        parameters={
            "hive_storage_format": "<to_be_edited>" #supported values are "csv", "parquet", "orc"
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_queue_data_source)

    explainability_result_data_source = DataSource(
        type="explain_result", 
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_DATABASE_NAME,
        table_name=EXPLAINABILITY_RESULT_TABLE_NAME,
        connection=DataSourceConnection(
            type="hive", 
            integrated_system_id=hive_integrated_system_id
        ),
        parameters={},
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_result_data_source)

In [22]:
# Adding the subscription

subscription_details = Subscriptions(wos_client).add(
    data_mart_id=data_mart_id,
    service_provider_id=service_provider_id,
    asset=asset,
    deployment=asset_deployment,
    asset_properties=asset_properties_request,
    analytics_engine=analytics_engine,
    scoring_endpoint=scoring_endpoint).result

subscription_id = subscription_details.metadata.id
print(subscription_details)

{
  "metadata": {
    "id": "e2df4ec7-6c75-416f-a444-8d21389f7513",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:e2df4ec7-6c75-416f-a444-8d21389f7513",
    "url": "/v2/subscriptions/e2df4ec7-6c75-416f-a444-8d21389f7513",
    "created_at": "2024-07-17T07:11:44.727000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "service_provider_id": "4d2f2fb2-6b64-4d58-8f13-257166e468e9",
    "asset": {
      "asset_id": "592b902d-3dc9-4e56-8bcb-86cbf1a6d8a9",
      "url": "https://internal-nginx-svc:12443/ml/v4/models/592b902d-3dc9-4e56-8bcb-86cbf1a6d8a9?space_id=088c142e-f35e-4e48-a30c-ad55a6edeecc&version=2020-06-12",
      "name": "gcr - P2 XGB Classifier - Model",
      "asset_type": "model",
      "problem_type": "binary",
      "input_data_type": "structured"
    },
    "asset_properties": {
      "label_column": "Risk",
      "prediction_field": "prediction",
   

In [23]:
import time
# Checking subscription status

state = wos_client.subscriptions.get(subscription_id).result.entity.status.state

while state not in ["active", "error"]:
    state = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    print(state)
    time.sleep(5)

preparing
active


In [24]:
# Add training, output, and input data schemas to the subscription
schemas_patch_document = [
    JsonPatchOperation(op=OperationTypes.REPLACE, path='/asset_properties/training_data_schema', value=common_configuration["training_data_schema"]),
    JsonPatchOperation(op=OperationTypes.REPLACE, path='/asset_properties/input_data_schema', value=common_configuration["input_data_schema"]),
    JsonPatchOperation(op=OperationTypes.REPLACE, path='/asset_properties/output_data_schema', value=common_configuration["output_data_schema"])
]

wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=schemas_patch_document)

# Add data_sources to the subscription
data_sources_patch_document=[
    JsonPatchOperation(op=OperationTypes.ADD, path='/data_sources', value=[data_source.to_dict() for data_source in data_sources])
]

wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=data_sources_patch_document)

In [25]:
import time

# Checking subscription status after the patch operation

state = wos_client.subscriptions.get(subscription_id).result.entity.status.state

while state not in ["active", "error"]:
    state = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    print(state)
    time.sleep(5)

preparing
preparing
preparing
preparing
preparing
preparing
preparing
preparing
preparing
preparing
active


# 4. Quality monitoring <a name="quality"></a>

### Enable the quality monitor

In the following code cell, default values are set for the quality monitor. You can change the default values by updating the optional `min_feedback_data_size` attribute in the `parameters` dict and set the quality threshold in the `thresholds` list.

In [26]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_feedback_data_size": 200
}

thresholds = [{
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
}]

quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

quality_monitor_instance_id = quality_monitor_details.metadata.id
print(quality_monitor_details)

{
  "metadata": {
    "id": "004e8a8d-2589-4a87-95e3-782a91fac214",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:004e8a8d-2589-4a87-95e3-782a91fac214",
    "url": "/v2/monitor_instances/004e8a8d-2589-4a87-95e3-782a91fac214",
    "created_at": "2024-07-17T07:13:05.844000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "e2df4ec7-6c75-416f-a444-8d21389f7513"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type": "relative",
        "delay_unit": "minute",
        "delay": 10
      },
      "rep

### Check monitor instance status

In [27]:
quality_status = None
from datetime import datetime

while quality_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
    quality_status = monitor_instance_details.entity.status.state
    if quality_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), quality_status)
        time.sleep(30)
        
print(datetime.utcnow().strftime('%H:%M:%S'), quality_status)

07:13:13 active


In [28]:
monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "004e8a8d-2589-4a87-95e3-782a91fac214",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:004e8a8d-2589-4a87-95e3-782a91fac214",
    "url": "/v2/monitor_instances/004e8a8d-2589-4a87-95e3-782a91fac214",
    "created_at": "2024-07-17T07:13:05.844000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-07-17T07:13:06.391000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "e2df4ec7-6c75-416f-a444-8d21389f7513"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "t

### Run an on-demand evaluation

Please make sure you have data in feedback table in hive. Quality evaluation will use this data for its metrics computation.

In [29]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "98c55159-a5b6-4c0e-9beb-1b21c5582116",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:98c55159-a5b6-4c0e-9beb-1b21c5582116",
    "url": "/v2/monitor_instances/9200be2d-ce34-4ca0-b7ca-ef82721fb31b/runs/98c55159-a5b6-4c0e-9beb-1b21c5582116",
    "created_at": "2024-07-17T10:52:04.252000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "last_processed_time": "2024-07-16T07:32:21.788398Z",
      "min_feedback_data_size": 10
    },
    "status": {
      "state": "running",
      "queued_at": "2024-07-17T10:52:04.238000Z",
      "started_at": "2024-07-17T10:52:04.252000Z",
      "operators": []
    }
  }
}


In [30]:
# Check run status

quality_run_status = None
while quality_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=quality_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    quality_run_status = monitoring_run_details.entity.status.state
    if quality_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), quality_run_status)
        time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), quality_run_status)

10:57:12 finished


### Display quality metrics

In [31]:
print(monitoring_run_details.entity)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

{
  "parameters": {
    "job_run_schedule_id": "2d41aa8b-a988-4188-9fc7-99725d90d765",
    "last_processed_time": "2024-07-16T07:32:21.788398Z",
    "min_feedback_data_size": 10,
    "total_records_processed": 10
  },
  "status": {
    "state": "finished",
    "queued_at": "2024-07-17T10:52:04.238000Z",
    "started_at": "2024-07-17T10:52:05.024000Z",
    "updated_at": "2024-07-17T10:57:06.816000Z",
    "completed_at": "2024-07-17T10:57:06.762000Z",
    "message": "Quality execution is completed.",
    "operators": [
      {
        "id": "original",
        "status": {
          "state": "finished",
          "completed_at": "2024-07-17T10:57:06.762000Z"
        }
      }
    ]
  }
}


# 5. Drift monitoring <a name="drift"></a>

### Enable the drift monitor

In the following code cell, type a path to the drift configuration tar ball.

In [32]:
wos_client.monitor_instances.upload_drift_model(
    model_path="drift_archive.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

In the following code cell, default values are set for the drift monitor. You can change the default values by updating the values in the `parameters` section. The `min_samples` parameter controls the number of minimum records required in an evaluation time window for drift analysis to be completed. The `drift_threshold` parameter sets the threshold in decimal format for the drift percentage to trigger an alert. The `train_drift_model` parameter controls whether to learn drift artefacts (drift model training and data constraints learning) by IBM Watson OpenScale via Spark job accessing training data table.

In [33]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.05,
    "train_drift_model": False
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
print(drift_monitor_details)

{
  "metadata": {
    "id": "bf710062-9d4b-4796-ae3f-79abeb43165d",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:bf710062-9d4b-4796-ae3f-79abeb43165d",
    "url": "/v2/monitor_instances/bf710062-9d4b-4796-ae3f-79abeb43165d",
    "created_at": "2024-07-17T11:38:05.978000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "e2df4ec7-6c75-416f-a444-8d21389f7513"
    },
    "parameters": {
      "drift_threshold": 0.05,
      "min_samples": 10,
      "train_drift_model": false
    },
    "thresholds": [
      {
        "metric_id": "drift_magnitude",
        "type": "upper_limit",
        "value": 0.5
      },
      {
        "metric_id": "predicted_accuracy",
        "type": "upper_limit",
        "value": 0.8
      },
      {
        "metric_id": "data_dr

### Check monitor instance status

In [34]:
drift_status = None

while drift_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
    drift_status = monitor_instance_details.entity.status.state
    if drift_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), drift_status)

11:38:13 active


### Run an on-demand evaluation
Please make sure you have data in payload table in hive. Drift evaluation will use this data for its metrics computation.

In [35]:
# Check Drift monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "bf710062-9d4b-4796-ae3f-79abeb43165d",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:bf710062-9d4b-4796-ae3f-79abeb43165d",
    "url": "/v2/monitor_instances/bf710062-9d4b-4796-ae3f-79abeb43165d",
    "created_at": "2024-07-17T11:38:05.978000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-07-17T11:38:07.122000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "e2df4ec7-6c75-416f-a444-8d21389f7513"
    },
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": "spark-3.4.2",
      "drift_thre

In [36]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "a883cc9c-6330-4256-a8a1-e674094d2db6",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:a883cc9c-6330-4256-a8a1-e674094d2db6",
    "url": "/v2/monitor_instances/bf710062-9d4b-4796-ae3f-79abeb43165d/runs/a883cc9c-6330-4256-a8a1-e674094d2db6",
    "created_at": "2024-07-17T11:38:16.240000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": "spark-3.4.2",
      "drift_threshold": 0.05,
      "min_samples": 10,
      "model_drift_enabled": true,
      "table_schema": {
        "fields": [
          {
            "length": 64,
            "metadata": {},
            "name": "scoring_id",
            "nullable": false,
            "type": "string

In [37]:
# Check run status

drift_run_status = None
while drift_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_run_status = monitoring_run_details.entity.status.state
    if drift_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), drift_run_status)
        time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), drift_run_status)

11:39:13 running
11:39:43 running
11:40:13 running
11:40:43 finished


### Display drift metrics

In [38]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2024-07-17 11:38:18.069880+00:00,data_drift_magnitude,5721e547-660a-498d-93e5-c547d80be628,0.0,None,0.1,[],drift,bf710062-9d4b-4796-ae3f-79abeb43165d,a883cc9c-6330-4256-a8a1-e674094d2db6,subscription,e2df4ec7-6c75-416f-a444-8d21389f7513
2024-07-17 11:38:18.069880+00:00,drift_magnitude,5721e547-660a-498d-93e5-c547d80be628,0.013636363636363669,None,0.05,[],drift,bf710062-9d4b-4796-ae3f-79abeb43165d,a883cc9c-6330-4256-a8a1-e674094d2db6,subscription,e2df4ec7-6c75-416f-a444-8d21389f7513
2024-07-17 11:38:18.069880+00:00,predicted_accuracy,5721e547-660a-498d-93e5-c547d80be628,0.8613636363636363,None,None,[],drift,bf710062-9d4b-4796-ae3f-79abeb43165d,a883cc9c-6330-4256-a8a1-e674094d2db6,subscription,e2df4ec7-6c75-416f-a444-8d21389f7513


# 6. Fairness monitoring <a name="fairness"></a>

### Enable the fairness monitor

The following code cell, will enable the fairness monitor.

In [39]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

fairness_statistics_json = None
with open("fairness_statistics.json", "r") as fp:
    fairness_statistics_json = json.load(fp)
    
parameters = fairness_statistics_json["parameters"]
thresholds = fairness_statistics_json["thresholds"]

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

fairness_monitor_instance_id = fairness_monitor_details.metadata.id
print(fairness_monitor_details)

{
  "metadata": {
    "id": "9b31235c-5d0a-45f0-8507-f589e15d7651",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:9b31235c-5d0a-45f0-8507-f589e15d7651",
    "url": "/v2/monitor_instances/9b31235c-5d0a-45f0-8507-f589e15d7651",
    "created_at": "2024-07-17T11:40:43.786000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "fairness",
    "target": {
      "target_type": "subscription",
      "target_id": "e2df4ec7-6c75-416f-a444-8d21389f7513"
    },
    "parameters": {
      "training_data_last_processed_time": "2024-06-23T08:55:38.198019Z",
      "features": [
        {
          "minority": [
            "female"
          ],
          "feature": "Sex",
          "metric_ids": [
            "statistical_parity_difference",
            "fairness_value"
          ],
          "majority": [
            "male"
          ],
          "t

### Check monitor instance status

In [40]:
fairness_state = fairness_monitor_details.entity.status.state

while fairness_state not in ("active", "error"):
    print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=fairness_monitor_instance_id).result
    fairness_state = monitor_instance_details.entity.status.state
    time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)

11:40:43 preparing
11:41:13 preparing
11:41:44 active


### Run an on-demand evaluation
Please make sure you have data in payload table in hive. Fairness evaluation will use this data for its metrics computation.

In [41]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "330005ca-b19a-494b-8b9a-b7a9595d8096",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:330005ca-b19a-494b-8b9a-b7a9595d8096",
    "url": "/v2/monitor_instances/9b31235c-5d0a-45f0-8507-f589e15d7651/runs/330005ca-b19a-494b-8b9a-b7a9595d8096",
    "created_at": "2024-07-17T11:41:44.329000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "favourable_class": [
        "No Risk"
      ],
      "features": [
        {
          "feature": "Sex",
          "majority": [
            "male"
          ],
          "metric_ids": [
            "statistical_parity_difference",
            "fairness_value"
          ],
          "minority": [
            "female"
          ],
          "threshold": 0.95
        },
        {
          "feature": "Age",
          "majority": [
            [
              26,
              75
            ]
          ],
          "metric_ids": [
            "stati

In [42]:
# Check run status

fairness_run_status = monitoring_run_details.entity.status.state
while fairness_run_status not in ("finished", "error"):
    print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=fairness_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    fairness_run_status = monitoring_run_details.entity.status.state
    time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)

11:49:31 running
11:50:02 finished


### Display fairness metrics

In [43]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2024-07-17 11:55:48.643147+00:00,fairness_value,d8c5c19d-0748-41a3-8be5-11b12ffe31cb,175.00000000000003,95.0,125.0,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,9b31235c-5d0a-45f0-8507-f589e15d7651,a653a968-db20-410e-a960-b752f88d56b1,subscription,e2df4ec7-6c75-416f-a444-8d21389f7513
2024-07-17 11:55:48.643147+00:00,statistical_parity_difference,d8c5c19d-0748-41a3-8be5-11b12ffe31cb,0.429,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,9b31235c-5d0a-45f0-8507-f589e15d7651,a653a968-db20-410e-a960-b752f88d56b1,subscription,e2df4ec7-6c75-416f-a444-8d21389f7513
2024-07-17 11:55:48.643147+00:00,fairness_value,d8c5c19d-0748-41a3-8be5-11b12ffe31cb,150.00000000000003,95.0,125.0,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,9b31235c-5d0a-45f0-8507-f589e15d7651,a653a968-db20-410e-a960-b752f88d56b1,subscription,e2df4ec7-6c75-416f-a444-8d21389f7513
2024-07-17 11:55:48.643147+00:00,statistical_parity_difference,d8c5c19d-0748-41a3-8be5-11b12ffe31cb,0.333,-0.15,0.15,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,9b31235c-5d0a-45f0-8507-f589e15d7651,a653a968-db20-410e-a960-b752f88d56b1,subscription,e2df4ec7-6c75-416f-a444-8d21389f7513
2024-07-17 11:46:47.141503+00:00,fairness_value,e70c8239-e175-485b-8dcb-1c5fd6785f65,175.00000000000003,95.0,125.0,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,9b31235c-5d0a-45f0-8507-f589e15d7651,330005ca-b19a-494b-8b9a-b7a9595d8096,subscription,e2df4ec7-6c75-416f-a444-8d21389f7513
2024-07-17 11:46:47.141503+00:00,statistical_parity_difference,e70c8239-e175-485b-8dcb-1c5fd6785f65,0.429,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,9b31235c-5d0a-45f0-8507-f589e15d7651,330005ca-b19a-494b-8b9a-b7a9595d8096,subscription,e2df4ec7-6c75-416f-a444-8d21389f7513
2024-07-17 11:46:47.141503+00:00,fairness_value,e70c8239-e175-485b-8dcb-1c5fd6785f65,150.00000000000003,95.0,125.0,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,9b31235c-5d0a-45f0-8507-f589e15d7651,330005ca-b19a-494b-8b9a-b7a9595d8096,subscription,e2df4ec7-6c75-416f-a444-8d21389f7513
2024-07-17 11:46:47.141503+00:00,statistical_parity_difference,e70c8239-e175-485b-8dcb-1c5fd6785f65,0.333,-0.15,0.15,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,9b31235c-5d0a-45f0-8507-f589e15d7651,330005ca-b19a-494b-8b9a-b7a9595d8096,subscription,e2df4ec7-6c75-416f-a444-8d21389f7513


# 7. Explainability monitoring <a name="explainability"></a>

### Enable the explainability monitor

#### Upload explainability configuration archive
In the following code cell, type the path to the explainability configuration archive tar ball.

In [44]:
with open("explainability.tar.gz", mode="rb") as explainability_tar:
    wos_client.monitor_instances.upload_explainability_archive(subscription_id=subscription_id, archive=explainability_tar)

print("Uploaded explainability archive successfully.")

Uploaded explainability archive successfully.


In [45]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
# Uncomment the below lines to enable lime global explanation. Available from Cloud Pak for Data version 4.6.4 onwards.
#    "global_explanation": {
#        "enabled": True,  # Flag to enable global explanation 
#        "explanation_method": "lime",
#        "sample_size": 1000, # [Optional] The sample size of records to be used for generating payload data global explanation. If not specified entire data in the payload window is used.
#    }
}

explainability_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_instance_id = explainability_monitor_details.metadata.id
print(explainability_monitor_details)


{
  "metadata": {
    "id": "c82f89f1-31fb-4572-a429-48a57c5b3e40",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:c82f89f1-31fb-4572-a429-48a57c5b3e40",
    "url": "/v2/monitor_instances/c82f89f1-31fb-4572-a429-48a57c5b3e40",
    "created_at": "2024-07-17T12:14:36.299000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "e2df4ec7-6c75-416f-a444-8d21389f7513"
    },
    "parameters": {},
    "thresholds": [
      {
        "metric_id": "global_explanation_stability",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "status": {
      "state": "preparing"
    }
  }
}


### Check monitor instance status

In [46]:
explainability_status = None

while explainability_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
    explainability_status = monitor_instance_details.entity.status.state
    if explainability_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)

04:59:02 active


### Run an on-demand evaluation
Please make sure you have data in Explain queue table in hive. Explainability will use this data for computing explanations.

In [47]:
# Check Explainbility monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "c82f89f1-31fb-4572-a429-48a57c5b3e40",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:c82f89f1-31fb-4572-a429-48a57c5b3e40",
    "url": "/v2/monitor_instances/c82f89f1-31fb-4572-a429-48a57c5b3e40",
    "created_at": "2024-07-17T12:14:36.299000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-07-17T12:17:46.510000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "e2df4ec7-6c75-416f-a444-8d21389f7513"
    },
    "parameters": {
      "config_modified_at": "2024-07-17T12:17:46.401452Z",
      "contrastive": {
        "enabled": true
      },
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "lime": {
        "enabled": true,
        "features_count": 10,
        "pe

In [48]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=explainability_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "92541a9d-8e65-411b-ba97-7e8e706f92c2",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:92541a9d-8e65-411b-ba97-7e8e706f92c2",
    "url": "/v2/monitor_instances/c82f89f1-31fb-4572-a429-48a57c5b3e40/runs/92541a9d-8e65-411b-ba97-7e8e706f92c2",
    "created_at": "2024-07-22T04:59:22.112000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "config_modified_at": "2024-07-17T12:17:46.401452Z",
      "contrastive": {
        "enabled": true
      },
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "lime": {
        "enabled": true,
        "features_count": 10,
        "perturbations_count": 5000
      },
      "validate_table_job_app_id": null,
      "validate_table_job_id": "666748cb-3f55-4cf2-9a58-80479c7ec80c",
      "validate_table_job_output_path": "explainability_configuration/51dfbd64-98f5-4ee4-8778-a6c21c28a8c9/output/a85cccc7-1ec5-49c3-a685-b

In [49]:
# Check run status

explainability_run_status = None
while explainability_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=explainability_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    explainability_run_status = monitoring_run_details.entity.status.state
    if explainability_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
        time.sleep(60)
        
print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)

05:01:46 running
05:02:46 running
05:03:47 running
05:04:47 finished


In [ ]:
# View the global explanation stability metric. When lime global explanation is enabled, the monitor run computes global explanation and publishes global_explanation_stability metric.
# wos_client.monitor_instances.show_metrics(monitor_instance_id=explainability_monitor_instance_id)

### Display sample explanations

In [50]:
explanations = wos_client.monitor_instances.get_all_explaination_tasks(subscription_id=subscription_id).result.to_dict()
print(explanations)

{'total_count': 12, 'limit': 50, 'offset': 0, 'explanation_fields': ['explanation_task_id', 'scoring_id', 'created_at', 'finished_at', 'status', 'prediction', 'subscription_id', 'deployment_id', 'asset_name', 'deployment_name', 'probability', 'explanation_type'], 'explanation_values': [['47682a56-766c-4a77-8cd1-76760c50e24a', 'sc1', '2024-07-02T05:10:33.192446Z', '2024-07-02T05:10:33.192859Z', 'finished', 'No Risk', '7585a078-c1b6-4a77-9ab3-87be9fb6026c', '2b976af0-e4ab-4859-af7d-2f2287d864ad', 'gcr - P2 XGB Classifier - Model', 'gcr model', 0.8429199, 'lime'], ['df3ebfcd-874c-4f7f-9aa2-c586a8daf5a7', 'sc2', '2024-07-02T05:10:33.200075Z', '2024-07-02T05:10:33.200286Z', 'finished', 'No Risk', '7585a078-c1b6-4a77-9ab3-87be9fb6026c', '2b976af0-e4ab-4859-af7d-2f2287d864ad', 'gcr - P2 XGB Classifier - Model', 'gcr model', 0.8291593, 'lime'], ['550f0274-65de-434f-b3b2-ff2c6a0f1150', 'sc6', '2024-07-02T05:10:33.221617Z', '2024-07-02T05:10:33.221821Z', 'finished', 'No Risk', '7585a078-c1b6-4a7

### Generate on demand Contrastive explanation

Choose any record from payload logging table or Explain Queue table and generate Lime, Contrastive explanations for the record. Please note that contrastive explanations are only supported for structured classification models. So specify the explanation_types accordingly. 

In [51]:
import time
 # Please enter scoring id(from payload table or Explain Queue table), for which you want to generate an explanation
 # scoring_ids = ["<scoring_id_from_pl_or_explain_queue_table>"]

# As an example, we will generate a contrastive explanation for the first scoring id from the list of explanations generated above
# as part of the Explainability monitor run
scoring_ids = [explanations["explanation_values"][0][1]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))

explanation_types = ["lime", "contrastive"]
explanation_task_ids = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, subscription_id=subscription_id, explanation_types=explanation_types).result.metadata.explanation_task_ids
explanation_task_id = explanation_task_ids[0]

print("Getting explanation for explanation task id: {}".format(explanation_task_id))
explanation = wos_client.monitor_instances.get_explanation_tasks(explanation_task_id, subscription_id=subscription_id).result
while explanation.entity.status.state not in ("finished", "error"):
        explanation = wos_client.monitor_instances.get_explanation_tasks(explanation_task_id, subscription_id=subscription_id).result
        explanation_run_status = explanation.entity.status.state
        if explanation_run_status not in ("finished", "error"):
            print(datetime.utcnow().strftime("%H:%M:%S"), explanation_run_status)
            time.sleep(60)
            
            
print(explanation)

Running explanations on scoring IDs: ['sc1']
Getting explanation for explanation task id: 9fc5d385-902c-40f0-822e-28df6f08faab
05:21:37 in_progress
05:22:38 in_progress
{
  "metadata": {
    "explanation_task_id": "9fc5d385-902c-40f0-822e-28df6f08faab",
    "created_by": "1000331001",
    "created_at": "2024-07-22T05:22:56.223426Z",
    "updated_at": "2024-07-22T05:22:56.223510Z"
  },
  "entity": {
    "status": {
      "state": "finished"
    },
    "asset": {
      "id": "592b902d-3dc9-4e56-8bcb-86cbf1a6d8a9",
      "name": "gcr - P2 XGB Classifier - Model",
      "input_data_type": "structured",
      "problem_type": "binary",
      "deployment": {
        "id": "2b976af0-e4ab-4859-af7d-2f2287d864ad",
        "name": "gcr model"
      }
    },
    "input_features": [
      {
        "name": "CheckingStatus",
        "value": "less_0",
        "feature_type": "categorical"
      },
      {
        "name": "LoanDuration",
        "value": 18,
        "feature_type": "numerical"
      

### Cleanup the untarred files

In [ ]:
import os
files = ["common_configuration.json", "explainability.tar.gz", "drift_archive.tar.gz", "fairness_statistics.json"]
for file in files:
    if os.path.isfile(file):
        os.remove(file)

## Congratulations!

You have finished the Batch demo for IBM Watson OpenScale by using Apache Spark on Cloud Pak for Data IBM Analytics Engine. You can now view the [Watson OpenScale Dashboard](https://url-to-your-cp4d-cluster/aiopenscale). Click the tile for the **German Credit model** to see the quality, drift and fairness monitors. Click the timeseries graph to get detailed information on transactions during a specific time window.